In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config("./config.json")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: azure-ml-pipeline
Azure region: eastus
Subscription id: 52c9f91d-c477-43c5-82bf-8c99b0dd262e
Resource group: azure-ml-pipeline


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found an existing cluster, using it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found an existing cluster, using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig 
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--C": uniform(0.05, 10),
        "--max_iter": choice(100, 200, 500, 1000)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create an environment for running the training script
env = Environment.from_pip_requirements('azure-ml-pipeline', 'requirements-train.txt')

print('env', env)

# Create a run config
run_config = ScriptRunConfig(source_directory=".",
                             script="train.py",
                             environment=env,
                             compute_target=cpu_cluster)

# Create a HyperDriveConfig using the run config, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=run_config,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=4,
    max_concurrent_runs=4
)

env Environment(Name: azure-ml-pipeline,
Version: None)


In [13]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [14]:
# import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print('here is the best run', best_run)
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print(parameter_values)

best_run.register_model(
    model_name="hyperdrive_best_model.pkl", 
    model_path="outputs/hyperdrive.joblib", 
    properties={'Accuracy': best_run_metrics['Accuracy']})

here is the best run Run(Experiment: udacity-project,
Id: HD_efa87aef-7917-4b14-b4df-95a69c95aaa6_3,
Type: azureml.scriptrun,
Status: Completed)
['--C', '6.994430992051935', '--max_iter', '1000']


Model(workspace=Workspace.create(name='azure-ml-pipeline', subscription_id='52c9f91d-c477-43c5-82bf-8c99b0dd262e', resource_group='azure-ml-pipeline'), name=hyperdrive_best_model.pkl, id=hyperdrive_best_model.pkl:1, version=1, tags={}, properties={'Accuracy': '0.9118079823432039'})

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(
    path=web_url, separator=",", header=True)

In [5]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

x_train['y'] = y_train
x_test['y'] = y_test


Here is ds TabularDataset
{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    debug_log="automl_errors.log",
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=cpu_cluster)

In [7]:
# Submit your automl run
automl_run = exp.submit(automl_config)

### YOUR CODE HERE ###

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_37843bb6-dd39-4a2b-bef5-f87568ed9778,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
# Retrieve and save your best automl model.

# Todo try doing this in the Azure environment

### YOUR CODE HERE ###
best_run, best_model = automl_run.get_output()
best_run.register_model(model_name = 'automl_best_model.pkl',  model_path = './outputs/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ERROR:azureml._file_utils.file_utils:Failed to download file with error: [Errno 13] Permission denied: 'C:\\Users\\verci\\AppData\\Local\\Temp\\tmpchpb9_br'


ModuleNotFoundError: No module named 'sklearn.impute._base'; 'sklearn.impute' is not a package

In [25]:
# Delete the cpu cluster

cpu_cluster.delete()